In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd -q ..

In [3]:
import lcdb
import json
import lcpfn 
import torch as th
from functools import partial
import numpy as np
from matplotlib import pyplot as plt

In [4]:
# path = "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\Year 1\\Q3\\Deep Learning\\deep-learning-project\\notebooks\data.json"

def readDatasetJson(path):
    f = open(path)

    dataset = json.load(f)
    # print(len(dataset))
    
    x = []
    y = []

    for v in dataset.values():
        x.append(v[0])
        y.append(v[1])
        
    x_true = th.Tensor(x)
    y_true = th.Tensor(y)
    
    return x_true, y_true


In [5]:
import os
def find_file(root_dir, target_filename):
    for root, dirs, files in os.walk(root_dir):
        if target_filename in files:
            return os.path.join(root, target_filename)
    
    # If the loop completes without returning, the file was not found
    return None

def find_data():
    return find_file(os.getcwd(), 'data.json')

In [6]:
def get_data():
    return readDatasetJson(find_data())

In [7]:
# function producing batches for PFN training
def get_batch(
    batch_size,
    seq_len,
    num_features,
    device="cpu",
    noisy_target=True,
    **_,
):
    assert num_features == 1

    x_data, y_data = get_data()
    x_data = x_data[:batch_size, :]
    y_data = y_data[:batch_size, :]

    y_data_noisy = y_data.clone()

    # print(x_data.shape)
    # print(y_data.shape)

    x_data = x_data.view((num_features, batch_size, seq_len)).transpose(2, 0).to(device)
    y_data = y_data.transpose(1, 0).to(device)
    y_data_noisy = y_data.clone()




    # x = np.arange(1, seq_len + 1)
    # y_target = np.empty((batch_size, seq_len), dtype=float)
    # y_noisy = np.empty((batch_size, seq_len), dtype=float)
    # x_data, y_data = get_data() # uses numpy rng
    
    # for i in range(batch_size):    
    #     if noisy_target:
    #         y_noisy[i] = y_data[i]
    #         y_target[i] = y_data[i]
    #     else:
    #         y_target[i], y_noisy[i] = y_data[i]
    # # turn numpy arrays into correctly shaped torch tensors & move them to device
    # x = (
    #     th.Tensor(x_data[0])
    #     .repeat((num_features, batch_size, 1))
    #     .transpose(2, 0)
    #     .to(device)
    # )
    # y_target = th.from_numpy(y_target).transpose(1, 0).to(device)
    # y_noisy = th.from_numpy(y_noisy).transpose(1, 0).to(device)

    # # changes
    # x = x.float()
    # y_target = y_target.float()
    # y_noisy = y_noisy.float()
    
    # return x, y_noisy, y_target

    return x_data, y_data, y_data_noisy

In [23]:
result = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=500,
                         epochs=20)

#th.save(result[2].state_dict(), "C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\deep-learning-project\\notebooks\\model.txt")
result[2].eval()

Using cpu:0 device
init dist
Not using distributed
DataLoader.__dict__ {'num_steps': 100, 'get_batch_kwargs': {'batch_size': 500, 'eval_pos_seq_len_sampler': <function train.<locals>.eval_pos_seq_len_sampler at 0x000002E8CE5910D0>, 'seq_len_maximum': 26, 'device': 'cpu:0', 'num_features': 1, 'hyperparameters': {}}, 'num_features': 1}
Style definition: None
Using a Transformer with 6.46 M parameters


c:\Users\mjaic\miniconda3\envs\lcpfn\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


TransformerModel(
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (linear2): Linear(in_features=512, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.2, inplace=False)
        (dropout2): Dropout(p=0.2, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=True)
        (dropout): Dro

In [22]:
result1 = lcpfn.train_lcpfn(get_batch_func=get_batch, 
                         seq_len=26,
                         emsize=256,
                         nlayers=12,
                         lr=0.001,
                         batch_size=0,
                         epochs=0)

result1[2].load_state_dict(th.load("C:\\Users\\mjaic\\OneDrive\\Desktop\\Delft\\deep-learning-project\\notebooks\\model.txt"))
print(result[2].state_dict())


Using cpu:0 device
init dist
Not using distributed
DataLoader.__dict__ {'num_steps': 100, 'get_batch_kwargs': {'batch_size': 0, 'eval_pos_seq_len_sampler': <function train.<locals>.eval_pos_seq_len_sampler at 0x000002E8CDE455E0>, 'seq_len_maximum': 26, 'device': 'cpu:0', 'num_features': 1, 'hyperparameters': {}}, 'num_features': 1}
Style definition: None
Using a Transformer with 6.46 M parameters
OrderedDict([('transformer_encoder.layers.0.self_attn.in_proj_weight', tensor([[-0.0487, -0.0664,  0.0481,  ...,  0.0518, -0.0009,  0.0243],
        [-0.0404, -0.0317,  0.0290,  ...,  0.0640,  0.0169,  0.0079],
        [ 0.0692, -0.0076, -0.0366,  ...,  0.0476, -0.0671, -0.0295],
        ...,
        [-0.0315, -0.0260, -0.0637,  ...,  0.0314,  0.0008, -0.0747],
        [-0.0649, -0.0102,  0.0120,  ...,  0.0250, -0.0662,  0.0672],
        [ 0.0097, -0.0424, -0.0389,  ...,  0.0373,  0.0396,  0.0677]])), ('transformer_encoder.layers.0.self_attn.in_proj_bias', tensor([-4.4457e-03, -4.8923e-03, -3.